In [1]:
import pandas as pd
credit_record = pd.read_csv("credit_record.csv")
credit_record.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


In [2]:
application_record = pd.read_csv("application_record.csv")
application_record.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [3]:
pt=pd.pivot_table(data=credit_record,index='ID',columns='STATUS',values='MONTHS_BALANCE',aggfunc='count',fill_value=0)
pt['total_delinquent']=pt.iloc[:,1]+pt.iloc[:,2]+pt.iloc[:,3]+pt.iloc[:,4]+pt.iloc[:,5]
pt['target']=pt['total_delinquent'].apply(lambda x:1 if x!=0 else 0)
pt.head()

STATUS,0,1,2,3,4,5,C,X,total_delinquent,target
ID,,,,,,,,,,
5001711,3,0,0,0,0,0,0,1,0,0
5001712,10,0,0,0,0,0,9,0,0,0
5001713,0,0,0,0,0,0,0,22,0,0
5001714,0,0,0,0,0,0,0,15,0,0
5001715,0,0,0,0,0,0,0,60,0,0


In [4]:
pt = pt[['target']]

In [5]:
pt.shape

(45985, 1)

In [6]:
credit_record.shape

(1048575, 3)

In [7]:
application_record.shape

(438557, 18)

In [8]:
final = pd.merge(pt,application_record,on='ID')
final.shape

(36457, 19)

In [9]:
final = final.drop('ID',axis=1)
final.head()

,target,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,1,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,1,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,0,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,0,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,0,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [10]:
import sweetviz as sv
sweet_report = sv.analyze(final)
sweet_report.show_html('sweet_report.html')


Report sweet_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [11]:
import pandas as pd
from pandas_profiling import ProfileReport
design_report = ProfileReport(final)
design_report.to_file(output_file='report.html')

In [33]:
import dtale
import pandas as pd
d = dtale.show(final)
d.open_browser()

In [11]:
X = final.drop('target',axis=1)
y = final['target']
from xverse.transformer import WOE
clf = WOE()
clf.fit(X, y)

WOE(mono_custom_binning={'AMT_INCOME_TOTAL': array([  27000.,  135000.,  202500., 1575000.]),
                         'CNT_CHILDREN': array([ 0.,  1., 19.]),
                         'CNT_FAM_MEMBERS': array([ 1.,  2., 20.]),
                         'DAYS_BIRTH': array([-25152., -18085., -13545.,  -7489.]),
                         'DAYS_EMPLOYED': array([-15713.,  -2512.,   -762., 365243.]),
                         'FLAG_EMAIL': array([0., 1.]),
                         'FLAG_MOBIL': array([1.]),
                         'FLAG_PHONE': array([0., 1.]),
                         'FLAG_...
                                  'Low-skill Laborers': 0.5550706919292876,
                                  'Managers': 0.10884426286411482,
                                  'Medicine staff': 0.15732562042824635,
                                  'NA': -0.11623544902968029,
                                  'Private service staff': -0.6691189041220243,
                                  'Realty age

In [12]:
X = clf.transform(X)
X

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,0.090813,-0.042655,-0.062117,-0.016592,0.092374,-0.017516,-0.012749,0.064983,0.191859,0.130236,0.011547,0.0,-0.000857,0.012689,-0.019294,-0.116235,-0.009197
1,0.090813,-0.042655,-0.062117,-0.016592,0.092374,-0.017516,-0.012749,0.064983,0.191859,0.130236,0.011547,0.0,-0.000857,0.012689,-0.019294,-0.116235,-0.009197
2,0.090813,-0.042655,-0.062117,-0.016592,-0.041559,-0.017516,-0.010246,-0.013198,-0.014551,-0.079759,0.031467,0.0,0.000250,0.012689,-0.019294,0.308644,-0.009197
3,-0.047125,0.025439,-0.062117,-0.016592,0.092374,0.088521,-0.010246,0.104669,-0.014551,-0.079759,0.011547,0.0,0.000250,-0.030863,0.181365,-0.059897,-0.009197
4,-0.047125,0.025439,-0.062117,-0.016592,0.092374,0.088521,-0.010246,0.104669,-0.014551,-0.079759,0.011547,0.0,0.000250,-0.030863,0.181365,-0.059897,-0.009197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36452,-0.047125,-0.042655,-0.062117,-0.016592,-0.046371,-0.017516,-0.010246,-0.013198,-0.014551,0.130236,0.031467,0.0,0.000250,0.012689,-0.019294,0.103968,0.024002
36453,-0.047125,-0.042655,-0.062117,-0.016592,-0.046371,-0.017516,-0.010246,-0.013198,-0.014551,0.130236,0.031467,0.0,0.000250,0.012689,-0.019294,0.103968,0.024002
36454,-0.047125,-0.042655,-0.062117,-0.016592,-0.046371,-0.017516,-0.010246,-0.013198,-0.014551,0.130236,0.031467,0.0,0.000250,0.012689,-0.019294,0.103968,0.024002
36455,-0.047125,-0.042655,-0.062117,-0.016592,-0.046371,-0.017516,-0.010246,-0.013198,-0.014551,0.130236,0.031467,0.0,0.000250,0.012689,-0.019294,0.103968,0.024002


In [13]:
clf.woe_df.head(50)

,Variable_Name,Category,Count,Event,Non_Event,Event_Rate,Non_Event_Rate,Event_Distribution,Non_Event_Distribution,WOE,Information_Value
0,AMT_INCOME_TOTAL,"(26999.999, 135000.0]",14473,1642,12831,0.113453,0.886547,0.382661,0.398899,-0.041559,4.030229e-03
1,AMT_INCOME_TOTAL,"(135000.0, 202500.0]",10702,1209,9493,0.112970,0.887030,0.281753,0.295125,-0.046371,4.030229e-03
2,AMT_INCOME_TOTAL,"(202500.0, 1575000.0]",11282,1440,9842,0.127637,0.872363,0.335586,0.305975,0.092374,4.030229e-03
3,CNT_CHILDREN,"(-0.001, 1.0]",32693,3792,28901,0.115988,0.884012,0.883710,0.898495,-0.016592,2.255834e-03
4,CNT_CHILDREN,"(1.0, 19.0]",3764,499,3265,0.132572,0.867428,0.116290,0.101505,0.135981,2.255834e-03
5,CNT_FAM_MEMBERS,"(0.999, 2.0]",26450,3088,23362,0.116749,0.883251,0.719646,0.726295,-0.009197,2.207454e-04
6,CNT_FAM_MEMBERS,"(2.0, 20.0]",10007,1203,8804,0.120216,0.879784,0.280354,0.273705,0.024002,2.207454e-04
7,CODE_GENDER,F,24430,2758,21672,0.112894,0.887106,0.642741,0.673755,-0.047125,4.278055e-03
8,CODE_GENDER,M,12027,1533,10494,0.127463,0.872537,0.357259,0.326245,0.090813,4.278055e-03
9,DAYS_BIRTH,"(-25152.001, -18085.0]",12155,1333,10822,0.109667,0.890333,0.310650,0.336442,-0.079759,9.205405e-03


In [14]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size= .20,random_state=20)
clf = WOE()
clf.fit(Xtrain, ytrain)
Xtrain = clf.transform(Xtrain)
Xtest = clf.transform(Xtest)